In [3]:
import os
import fitz
import pathlib
import sys
import pandas as pd
from tqdm import tqdm

from pdf_processing import *
from llm_callbacks import anthropic_callback, openai_callback
from parser import SimpleTextRetreiver, PDFLLMParser

In [4]:
prompt_path = "./prompts/complexity_prompt.txt"
# directory = "./books/OpenStax/"
directory = "/Users/igorprotsenko/Desktop/books/"
# directory = "./data/resized_textbooks/"
write_path = '../data/processed_output/gpt_4_ouput/'
visualize_path = '../visualized/'

In [5]:

os.environ["openai_api_key"] = "..."
os.environ["anthropic_api_key"] = "..."


In [11]:
import os
prompt = pathlib.Path(prompt_path).read_text()

anthropic_func = anthropic_callback(prompt, model_version='claude-3-sonnet-20240229')
# anthropic_func = anthropic_callback(prompt, model_version='claude-3-haiku-20240307')

# openai_func = openai_callback(prompt, model_version='gpt-3.5-turbo-0125')
openai_func = openai_callback(prompt, model_version='gpt-4-0125-preview')

In [12]:
parser = PDFLLMParser(directory, write_path, visualize_path, openai_func)

In [13]:
all_pdfs = list(parser._scan_directory_for_pdfs())

In [14]:
path  =  list(parser._scan_directory_for_pdfs())[7]

In [15]:
visualized = parser._plot_bbs(path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 375.41it/s]


In [17]:
print(visualized)

../visualized/books_Script_PHYS200_fluids.pdf


In [16]:
write_path = parser.analyze_files(path)

Writing llm output to ../data/processed_output/gpt_4_ouput/Script_PHYS200_fluids/scores


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:42<00:00,  1.32s/it]

Finish doing llm inderence


In [18]:
print(write_path)

../data/processed_output/gpt_4_ouput/Script_PHYS200_fluids/scores


In [20]:
metadata = parser.aggregate_metadata(write_path)

In [23]:
print(metadata)

../data/processed_output/gpt_4_ouput/Script_PHYS200_fluids/scores/metadata.csv


In [25]:
pd.read_csv(metadata)

,file_path,file_name,page_number,score
0,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,29,0.40
1,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,15,0.70
2,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,14,0.70
3,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,28,0.55
4,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,16,0.35
5,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,17,0.60
6,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,13,0.65
7,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,12,0.55
8,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,10,0.50
9,../data/processed_output/gpt_4_ouput/Script_PH...,Script_PHYS200_fluids,11,0.55


In [26]:
print("Parsing easy examples")
parser.parse_pdfs(metadata, threshold=.3)

Parsing easy examples


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 104.02it/s]


PosixPath('../data/processed_output/gpt_4_ouput/Script_PHYS200_fluids/parsed_text/easy')

In [27]:
print("Parsing difficult examples")
parser.parse_pdfs(metadata, threshold=.4, is_greater=True)

Parsing difficult examples


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 118.65it/s]


PosixPath('../data/processed_output/gpt_4_ouput/Script_PHYS200_fluids/parsed_text/complex')